In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import ray
from ray import tune
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error 
from sklearn.model_selection import KFold
import torch

In [ ]:
#Loading training set into dataframe
df = pd.read_csv('./data/TOC_WF_LMX_2K.csv')
df.head()

In [ ]:
df = df.drop(columns='Depth/Thickness(m)')
df = df.drop(columns='Journal')
df = df.drop(columns='Author')
df = df.drop(columns='Well')
df = df.drop(columns='Area')
df = df.drop(columns='DOI1')
df = df.drop(columns='DOI2')
df = df.drop(columns='Unnamed: 11')
df = df.drop(columns='Unnamed: 12')

In [ ]:
#This is an example, taking TOC as label to predict
label_data = df['TOC(%)']
train_data = df.drop('TOC(%)', axis=1) # we don't need it in this project
label_data.shape, train_data.shape

In [ ]:
#Function to min-max normalize
def normalize(df, cols):
    """
    @param df pandas DataFrame
    @param cols a list of columns to encode
    @return a DataFrame with normalized specified features
    """
    result = df.copy() # do not touch the original df
    for feature_name in cols:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        if max_value > min_value:
            result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [ ]:
#Normalizing dataset
new_train = normalize(train_data,train_data.columns)
new_train

In [ ]:
label_data.isnull().values.any()
new_train.isnull().values.any()

In [ ]:
#Test Nan and fill with mean
for column in list(new_train.columns[ new_train.isnull().sum() > 0]):
    mean_val = new_train[column].mean()
    new_train[column].fillna(mean_val, inplace=True)

In [ ]:
training = np.array(new_train)
labeling = np.array(label_data)

In [ ]:
training = torch.tensor(training, dtype=torch.float32)
labeling = torch.tensor(labeling, dtype=torch.float32)

In [ ]:
labeling.shape, training.shape

In [ ]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [ ]:
# from utils.half_attenuate import HiddenSizesGenerator

def get_hidden_sizes(input_size):

    # Define the half-attenuation law of the neuron size

    hidden_sizes = []
    size = input_size
    while size > 16:
        hidden_sizes.append(size)
        size = size // 2
    return hidden_sizes

class WellLogNet(nn.Module):
    def __init__(self, hyperparams):
        super().__init__()

        # Fix the hyper-paremeters in the input and output layers
        
        if not isinstance(hyperparams, dict):
            hyperparams = {}

        self.input_dim = hyperparams["input_dim"]
        self.output_dim = hyperparams["output_dim"]

        layers = []
        in_size = hyperparams["input_dim"]
        out_size = hyperparams["output_dim"]
        sizes = get_hidden_sizes(hyperparams["hidden_sizes"])
        
        for i, size in enumerate(sizes[:-1]):
            layers.append(nn.Linear(in_size, size))
            in_size = size
        
        layers.append(nn.Linear(in_size, out_size))

        self.layers = nn.Sequential(*layers)
    
    def get_input_dim(self):
        return self.input_dim
    
    def get_output_dim(self):
        return self.output_dim

    def forward(self, x):

        # Here, we use relu function to activate except for the last layer

        for layer in self.layers[:-1]:
            x = F.relu(layer(x))
        return self.layers[-1](x)

class RONet(nn.Module):
    def __init__(self, hyperparams={}):
        super().__init__()

        # Fix the hyper-paremeters in the input and output layers
        
        if not isinstance(hyperparams, dict):
            hyperparams = {}
        
        self.input_dim = hyperparams["input_dim"]
        self.output_dim = hyperparams["output_dim"]

        layers = []
        in_size = hyperparams["input_dim"]
        out_size = hyperparams["output_dim"]
        sizes = get_hidden_sizes(hyperparams["hidden_sizes"])

        for i, size in enumerate(sizes[:-1]):
            layers.append(nn.Linear(in_size, size))
            in_size = size

        layers.append(nn.Linear(in_size, out_size))

        self.layers = nn.Sequential(*layers)

    def forward(self, x):

        # Here, we use relu function to activate except for the last layer

        for layer in self.layers[:-1]:
            x = F.relu(layer(x))
        return self.layers[-1](x)
    
class BiasNet(nn.Module):
    def __init__(self, hyperparams):
        super().__init__()

        # Fix the hyper-paremeters in the input and output layers
        
        if not isinstance(hyperparams, dict):
            hyperparams = {}
        
        self.input_dim = hyperparams["input_dim"]
        self.output_dim = hyperparams["output_dim"]
        
        layers = []
        in_size = hyperparams["input_dim"]
        out_size = hyperparams["output_dim"]
        sizes = get_hidden_sizes(hyperparams["hidden_sizes"])

        for i, size in enumerate(sizes[:-1]):
            layers.append(nn.Linear(in_size, size))
            in_size = size
        
        layers.append(nn.Linear(in_size, out_size))

        self.layers = nn.Sequential(*layers)
        
    def forward(self, x):

        # Here, we use relu function to activate except for the last layer

        for layer in self.layers[:-1]:
            x = F.relu(layer(x))
        return self.layers[-1](x)

class PasseyNet(nn.Module):
    def __init__(self, well_log_net, ro_net, bias_net):
        super().__init__()
        
        # 获取子网络的输入输出维度
        self.well_log_input_dim = well_log_net.input_dim
        self.well_log_output_dim = well_log_net.output_dim
        
        self.ro_input_dim = ro_net.input_dim
        self.ro_output_dim = ro_net.output_dim
        
        self.bias_input_dim = bias_net.input_dim
        self.bias_output_dim = bias_net.output_dim

        # 网络层
        self.well_log_net = well_log_net
        self.ro_net = ro_net
        self.bias_net = bias_net
        
        # 输入输出维度
        self.input_dim = self.well_log_input_dim
        self.output_dim = self.bias_output_dim 

    def forward(self, x):
        well_log_matrix = self.well_log_net(x)
        ro_matrix = self.ro_net(x)
        bias_matrix = self.bias_net(x)
        
        output = torch.matmul(torch.matmul(x, well_log_matrix.T), 
                              torch.pow(10.0, ro_matrix)) + bias_matrix
        return output

In [ ]:
# Set the hyperparameters for search in the Net 
config = {
  "well_log_net": {
      "input_dim": 3,
     "hidden_sizes": tune.choice([256, 512, 1024]),
     "output_dim": 3 
  },
  
  "ro_net": {
      "input_dim": 3,
     "hidden_sizes": tune.choice([256, 512, 1024]),
     "output_dim": 1
  },
  
  "bias_net": {
      "input_dim": 3,
     "hidden_sizes": tune.choice([256, 512, 1024]),
     "output_dim": 1
  }
}

In [ ]:
torch.autograd.set_detect_anomaly(True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

# Define the NAS by ray
def train_model(config):
    
    # Get config for each subnetwork
    well_log_config = config["well_log_net"]
    ro_config = config["ro_net"] 
    bias_config = config["bias_net"]
    
    # Initialize subnetworks
    well_log_net = WellLogNet(well_log_config)
    ro_net = RONet(ro_config)
    bias_net = BiasNet(bias_config) 

    # Initialize PasseyNet
    model = PasseyNet(well_log_net, ro_net, bias_net)
    
    model.to(device)

    x = training.to(device)
    y = labeling.to(device)

    # Initializing optimizer and loss functions
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    # Store loss、mae、mse、R2、mape
    loss_list = []
    mae_list = []
    mse_list = []
    r2_list = []
    r2_adjust_list = []
    mape_list = []

    for i in range(100):

        kf = KFold(n_splits=5, shuffle=True, random_state=42) # 5-fold

        for fold ,(train_idx, val_idx) in enumerate(kf.split(x)):

            # Dividing training and validating datasets
            train_x, train_y = x[train_idx], y[train_idx]
            val_x, val_y = x[val_idx], y[val_idx]

            for epoch in range(100):

                output = model(train_x) # Training
                output = output.cpu() # Return output to CPU
                label = train_y.cpu() # Return label to CPU
                
                loss = criterion(output, label)
                optimizer.zero_grad()   
                loss.backward()        
                optimizer.step()

                # Validating
                with torch.no_grad():
                    val_output = model(val_x)
                    val_output = val_output.cpu()
                    val_y = val_y.cpu()
                    val_loss = criterion(val_output, val_y)

                    val_pred = val_output.detach().numpy()
                    val_true = val_y.detach().numpy()

                    val_mae = mean_absolute_error(val_true, val_pred)
                    val_mse = mean_squared_error(val_true, val_pred)

                    val_r2 = r2_score(val_true, val_pred)
                    val_mape = mean_absolute_percentage_error(val_true, val_pred)

                    val_adjust_r2 = 1-((1-val_r2)*(len(val_x)-1))/(len(val_x)-6-1)
                    
                    # 打印结果  
                    # print(f'Epoch: {epoch+1:02d}, Loss: {loss:.4f}, R2: {val_r2:.4f}, MAE: {val_mae:.4f}, MSE: {val_mse:.4f}, MAPE: {val_mape:.4f}')
            
                # 存储loss、mae和mse
                loss_list.append(val_loss.item())
                mae_list.append(val_mae)
                mse_list.append(val_mse)
                r2_list.append(val_r2)
                r2_adjust_list.append(val_adjust_r2)
                mae_list.append(val_mape)
            
            # Plot the 5-fold training results        
            plt.plot(loss_list)
            plt.title('Five-Fold Loss Curve')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.show()

            # 绘制MAE、MSE、MAPE curves
            plt.plot(mae_list, label='MAE')
            plt.plot(mse_list, label='MSE')
            plt.plot(mae_list, label='MAE')
            plt.title('MAE, MSE, MAPE Curve')  
            plt.xlabel('Epoch')
            plt.ylabel('Error')
            plt.legend()
            plt.show()

            # Plot R2、R2_adjust curves
            plt.plot(r2_list, label='R2')
            plt.plot(r2_adjust_list, label='R2_adjust')
            plt.title('R2 & Adjusted R2 Curve')
            plt.xlabel('Epoch')
            plt.ylabel('Error')
            plt.legend()
            plt.show()
    
    score = -val_mse(model)

    return score

# Run tune
ray.shutdown()
ray.init(num_gpus=1)

analysis = tune.run(
    train_model,
    stop={"episode_reward_mean": 0.005},
    config=config,
    num_samples=10, # Number of tray
    metric="score",
    mode="max",
    resources_per_trial={"gpu": 1},
    fail_fast="raise"  
)

# Best config   
best_config = analysis.get_best_config()

# Generate model by the best config
well_log_net = WellLogNet(best_config["well_log_net"])
ro_net = RONet(best_config["ro_net"])
bias_net = BiasNet(best_config["bias_net"])
best_model = PasseyNet(well_log_net, ro_net, bias_net)


In [ ]:
#Loading training set into dataframe
test_df = pd.read_csv('./data/TOC_WF_LMX.csv')
test_df.head()

In [ ]:
test_df = test_df.drop(columns='Depth/Thickness(m)')
test_df = test_df.drop(columns='Journal')
test_df = test_df.drop(columns='Author')
test_df = test_df.drop(columns='Well')
test_df = test_df.drop(columns='Area')
test_df = test_df.drop(columns='DOI1')
test_df = test_df.drop(columns='DOI2')
test_df = test_df.drop(columns='Unnamed: 11')
test_df = test_df.drop(columns='Unnamed: 12')

In [ ]:
test_df.shape

In [ ]:
test_df_last_30 = test_df.tail(30)
test_df_last_30.shape

In [ ]:
#This is an example, taking TOC as label to predict
test_label = test_df_last_30['TOC(%)']
test_train = test_df_last_30.drop('TOC(%)', axis=1) # we don't need it in this project
test_label.shape, test_train.shape

In [ ]:
# test_train = test_train.drop(columns='井名')
# test_train = test_train.drop(columns='Depth')

In [ ]:
#Normalizing dataset
normalized_test_train = normalize(test_train, test_train.columns)
normalized_test_train

In [ ]:
#Test Nan and fill with mean
for column in list(normalized_test_train.columns[ normalized_test_train.isnull().sum() > 0]):
    mean_val = normalized_test_train[column].mean()
    normalized_test_train[column].fillna(mean_val, inplace=True)

In [ ]:
# 需要传入Tensor 
X_test = normalized_test_train.values # 转为numpy数组
X_test = torch.Tensor(X_test).to(device) # 转为Tensor


# 切换到训练模式
model.train() 

# 预测值 
y_pred = model(X_test)
y_pred = y_pred.cpu().detach().numpy()

# 真实值
y_true = test_label 

# 残差 
y_true = y_true.ravel()  
y_pred = y_pred.ravel()
resid = y_true - y_pred

# 绘图
plt.hist(resid, bins=20)
plt.title('Residuals Histogram')
plt.xlabel('Prediction Error')
plt.ylabel('Frequency')

# 均值和标准差
mean = resid.mean()
stddev = resid.std()

# 绘制竖直线
plt.axvline(mean, color='r')
plt.axvline(mean + 2*stddev, color='r', linestyle='--')
plt.axvline(mean - 2*stddev, color='r', linestyle='--')

plt.show()

In [ ]:
# 绘制预测值实际值差距图

# well_name = test_df_last_30['井名']

x = range(len(y_pred))

plt.scatter(x, y_pred, label='Predicted')
plt.scatter(x, y_true, label='True')

plt.title('Prediction-True Comparison')
plt.ylabel('TOC')
plt.xlabel('Sample Number')
plt.legend()
plt.show()

In [ ]:
plt.scatter(y_true, y_pred, label='Predicted')
plt.ylabel('Predicted TOC')
plt.xlabel('True TOC')

plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'k--', lw=2)
plt.title('Prediction-True Comparison')
plt.legend()
plt.show()